# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
#/stock/{symbol}/stats/{stat?}
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
r = requests.get(api_url)
if r.status_code < 200 or r.status_code >= 300:
    raise Error(r.reason)
# print(r.json())
data = r.json()

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data

{'companyName': 'Apple Inc',
 'marketcap': 2280897974307,
 'week52high': 136.73,
 'week52low': 56.64,
 'week52change': 0.8574692230757328,
 'sharesOutstanding': 17672800270,
 'float': 0,
 'avg10Volume': 119267740,
 'avg30Volume': 108006436,
 'day200MovingAvg': 111.17,
 'day50MovingAvg': 127.61,
 'employees': 0,
 'ttmEPS': 3.29,
 'ttmDividendRate': 0.8110428661594139,
 'dividendYield': 0.006314753678734597,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-30',
 'nextEarningsDate': '0',
 'peRatio': 40.965172469319256,
 'beta': 1.2044783973082327,
 'maxChangePercent': 51.6579223390038,
 'year5ChangePercent': 4.279634685096702,
 'year2ChangePercent': 2.769161556940965,
 'year1ChangePercent': 0.8963358407212866,
 'ytdChangePercent': 0.8470896262604225,
 'month6ChangePercent': 0.491266780321083,
 'month3ChangePercent': 0.2279085261841268,
 'month1ChangePercent': 0.14928628562325802,
 'day30ChangePercent': 0.14870561820208048,
 'day5ChangePercent': 0.0266030740088586}

In [5]:
data["year1ChangePercent"]

0.8963358407212866

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_strings = []
for stockSubList in chunks(stocks, 100):
    symbol_strings.append( ",".join(list(stockSubList["Ticker"])) )

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)
api_url_template = 'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={0}&types=stats,price&token={1}'
for symbol_string in symbol_strings:
    api_url = api_url_template.format(symbol_string, IEX_CLOUD_API_TOKEN)
    r = requests.get(api_url)
    if r.status_code < 200 or r.status_code >= 300:
        raise Error (r.reason)
    data = r.json()
    for key, value in  data.items():
        quoteData = value["stats"]
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    key,
                    value["price"],
                    quoteData["year1ChangePercent"],
                    "N/A"
                ],
                index = my_columns
            )
            , ignore_index = True
        )

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values(my_columns[2], ascending=False, inplace=True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop(["index"], axis=1, inplace=True)

In [9]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.88,2.2621,N/A
1,NVDA,538.80,1.22979,N/A
2,LB,40.35,1.19859,N/A
3,PYPL,241.98,1.19816,N/A
4,ALB,153.81,1.15526,N/A
5,AMD,95.92,0.993571,N/A
6,NOW,561.88,0.978762,N/A
7,CDNS,136.00,0.969475,N/A
8,FCX,25.80,0.935304,N/A
9,WST,293.50,0.926589,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [10]:
def portfolio_input():
    while(True):
        try:
            portfolio_size = float(input("Enter the size of your portfolio"))
            return portfolio_size
        except ValueError as e:
            print("Please enter a number")
        except Error as e:
            print(e)
        

In [11]:
portfolio_size = portfolio_input()

Enter the size of your portfolio1000000


In [12]:
position_size = portfolio_size / len(final_dataframe)
for cnt in range(len(final_dataframe)):
    final_dataframe.iloc[cnt, -1] = math.floor(position_size / final_dataframe.iloc[cnt, 1])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,38.88,2.2621,504
1,NVDA,538.80,1.22979,36
2,LB,40.35,1.19859,485
3,PYPL,241.98,1.19816,81
4,ALB,153.81,1.15526,127
5,AMD,95.92,0.993571,204
6,NOW,561.88,0.978762,34
7,CDNS,136.00,0.969475,144
8,FCX,25.80,0.935304,759
9,WST,293.50,0.926589,66


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [13]:
hqm_columns = [
    "Ticker",
    "Price",
    "Number of shares to buy",
    "One-Year Price Return",
    "One-Year Return Percentile",
    "Six-Month Price Return",
    "Six-Month Return Percentile",
    "Three-Month Price Return",
    "Three-Month Return Percentile",
    "One-Month Price Return",
    "One-Month Return Percentile"
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
api_url_template = 'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={0}&types=stats,price&token={1}'
for symbol_string in symbol_strings:
    api_url = api_url_template.format(symbol_string, IEX_CLOUD_API_TOKEN)
    r = requests.get(api_url)
    if r.status_code < 200 or r.status_code >= 300:
        raise Error (r.reason)
    data = r.json()
    for key, value in  data.items():
        quoteData = value["stats"]
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    key,
                    value["price"],
                    "N/A",
                    quoteData["year1ChangePercent"],
                    "N/A",
                    quoteData["month6ChangePercent"],
                    "N/A",
                    quoteData["month3ChangePercent"],
                    "N/A",
                    quoteData["month1ChangePercent"],
                    "N/A"
                ],
                index = hqm_columns
            )
            , ignore_index = True
        )

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [14]:
time_periods = [
    "One-Year",
    "Six-Month",
    "Three-Month",
    "One-Month"
]
#Fix any missing price return data
for time_period in time_periods:
    change_col = f"{time_period} Price Return"
    hqm_dataframe[change_col].fillna(0, inplace = True)

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f"{time_period} Price Return"
        percentile_col = f"{time_period} Return Percentile"
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
hqm_dataframe

,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,117.79,N/A,0.402013,0.863366,0.374774,0.70495,0.228819,0.588119,0.023927,0.625743
1,AAL,16.16,N/A,-0.476916,0.0178218,0.203419,0.4,0.341061,0.79604,0.058790,0.79604
2,AAP,163.33,N/A,0.008580,0.4,0.127872,0.277228,0.100479,0.318812,0.074968,0.845545
3,AAPL,135.00,N/A,0.888107,0.976238,0.477202,0.821782,0.231813,0.594059,0.151435,0.960396
4,ABBV,105.75,N/A,0.220718,0.69901,0.116228,0.255446,0.220928,0.568317,-0.006859,0.425743
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,108.98,N/A,0.096167,0.528713,0.255718,0.510891,0.184804,0.473267,0.003421,0.493069
501,ZBH,152.41,N/A,-0.001312,0.374257,0.276680,0.544554,0.139571,0.40198,-0.010802,0.40396
502,ZBRA,388.96,N/A,0.501577,0.918812,0.523908,0.859406,0.525564,0.942574,0.009486,0.538614
503,ZION,45.32,N/A,-0.129491,0.221782,0.343953,0.665347,0.569588,0.954455,0.035794,0.70297


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [15]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [16]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [17]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
hqm_dataframe[hqm_columns[2]] = position_size / hqm_dataframe['Price']
hqm_dataframe[hqm_columns[2]] = hqm_dataframe[hqm_columns[2]].apply(lambda x: math.floor(x))
hqm_dataframe

<ipython-input-17-ae5df181f548>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe[hqm_columns[2]] = position_size / hqm_dataframe['Price']
<ipython-input-17-ae5df181f548>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe[hqm_columns[2]] = hqm_dataframe[hqm_columns[2]].apply(lambda x: math.floor(x))


,Ticker,Price,Number of shares to buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.79,166,0.402013,0.863366,0.374774,0.70495,0.228819,0.588119,0.023927,0.625743,0.695545
1,AAL,16.16,1213,-0.476916,0.0178218,0.203419,0.4,0.341061,0.79604,0.058790,0.79604,0.502475
2,AAP,163.33,120,0.008580,0.4,0.127872,0.277228,0.100479,0.318812,0.074968,0.845545,0.460396
3,AAPL,135.00,145,0.888107,0.976238,0.477202,0.821782,0.231813,0.594059,0.151435,0.960396,0.838119
4,ABBV,105.75,185,0.220718,0.69901,0.116228,0.255446,0.220928,0.568317,-0.006859,0.425743,0.487129
5,ABC,101.74,192,0.155573,0.607921,-0.011292,0.0712871,0.029463,0.144554,-0.059782,0.0910891,0.228713
6,ABMD,311.71,62,0.768316,0.964356,0.261255,0.518812,0.163377,0.451485,0.144161,0.954455,0.722277
7,ABT,111.89,175,0.262491,0.758416,0.250839,0.50297,0.078824,0.271287,0.006989,0.518812,0.512871
8,ACN,269.00,72,0.234936,0.714851,0.285771,0.554455,0.211787,0.540594,0.029337,0.675248,0.621287
9,ADBE,509.37,38,0.533303,0.924752,0.163502,0.326733,0.071891,0.249505,0.083099,0.863366,0.591089


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [18]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [19]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [20]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [21]:
writer.save()